<a href="https://colab.research.google.com/github/kenrp8/Coursera_Capstone/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Import libraries
import folium
from geopy.geocoders import Nominatim
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML 
from sklearn.cluster import DBSCAN


##This section contains the code from the second part of this assignment

In [21]:
# Read the CVS coordenates data from given URL
post_code_coord = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_data.cvs')
#print('data shape:', post_code_coord.shape)

In [22]:
# Now we are interested in get the Postal Code info from Wikipedia

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html_doc=requests.get(url).text

#print(html_doc) #this print the raw html

soup = BeautifulSoup(html_doc, 'html.parser')

#To get the table, We can use the given hint for this assignment
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
  cell = {}
  if row.span.text=='Not assigned':
    pass
  else:
    cell['PostalCode'] = row.p.text[:3]
    cell['Borough'] = (row.span.text).split('(')[0]
    cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
    table_contents.append(cell)

#print(table_contents)
#Parse table into a Data Frame

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({
    'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
    'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
    'EtobicokeNorthwest':'Etobicoke Northwest',
    'East YorkEast Toronto':'East York/East Toronto',
    'MississaugaCanada Post Gateway Processing Centre':'Mississauga'
})

In [23]:
#Rename columns in the first dataframe
post_code_coord.columns = ['PostalCode', 'Latitude', 'Longitude']

#merge df using PostalCode reference
postal_code_info = pd.merge(df, post_code_coord, on='PostalCode')


Select Data from Postal Codes that are placed in Toronto

In [24]:
toronto_pc = postal_code_info[postal_code_info['Borough'].str.contains("Toronto", case=True)]
toronto_pc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [25]:
toronto_pc.shape

(39, 5)

##Exploring Neighborhoods

In [26]:
CLIENT_ID = 'C1PYNRX2W0BXFDN3LQFP3C300ZK34QGQA0KFMIFA1BUG14TW' # your Foursquare ID
CLIENT_SECRET = 'U1BII1E2GJKJ2MTMRCEL2NSDDMASNSN4LVAJ4CCVL1A5JP4A' # your Foursquare Secret
ACCESS_TOKEN = 'I3SXDJ245P5ANG0UFL2DVRGQG4D3KIF12NJTDZH45GXZOBSS' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100 # A default Foursquare API limit value

We can use de funtion used in New York venue analysis.

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We select the vunues placed in Toronto

In [28]:
toronto_venues = getNearbyVenues(
    names=toronto_pc['Neighborhood'],
    latitudes=toronto_pc['Latitude'],
    longitudes=toronto_pc['Longitude'],
)

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

Let's check the size of the resulting dataframe


In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(1579, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


For this case we want to know how many vunues are placed in each Neighborhood:

In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,64,64,64,64,64,64
Christie,16,16,16,16,16,16
Church and Wellesley,74,74,74,74,74,74
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33
Davisville North,6,6,6,6,6,6


Now we proceed to encode the data for clustering


In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
sort_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[sort_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


It is necessary to regroup the data using __Neighborhood__ key. The next step is normalize the data for clustering.

In [32]:
toronto_venue_frecuency = toronto_onehot.groupby('Neighborhood').mean().reset_index()


In [33]:
toronto_venue_frecuency.shape

(39, 235)

We will use the most frecuency venues in each Neighborhood to know how similar are the Neighborhoods.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_venue_frecuency['Neighborhood']

for ind in np.arange(toronto_venue_frecuency.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_venue_frecuency.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Cheese Shop,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Belgian Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Bar,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Plane,Harbor / Marina
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Thai Restaurant,Burger Joint,Korean Restaurant,Indian Restaurant
4,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub,Candy Store


We procees to clustering the data. We can see that there are many zero values. In this case we use __Density-based spatial clustering__ instead od k-mean.

In [36]:
# set number of clusters
toronto_venue_frecuency_cluster = toronto_venue_frecuency.drop('Neighborhood', 1)

# run dbscan clustering
dbscan = DBSCAN(eps=0.4, min_samples=1).fit(toronto_venue_frecuency_cluster)

# check cluster labels generated for each row in the dataframe
clusters_number = len(set(dbscan.labels_))

Now we can see the similarity between clusters e.g. The 0-label cluster elements have caffe shop as the most pupular venue.

In [37]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', dbscan.labels_)

clustered_toronto_pc = toronto_pc

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
clustered_toronto_pc = clustered_toronto_pc.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

clustered_toronto_pc.head(39)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Hotel,Ramen Restaurant,Pizza Place,Burger Joint
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cosmetics Shop,Cocktail Bar,Gym,Lingerie Store,Farmers Market,Seafood Restaurant,Restaurant,Italian Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,7,Health Food Store,Trail,Pub,Wings Joint,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Donut Shop
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Cheese Shop,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Belgian Restaurant
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Thai Restaurant,Burger Joint,Korean Restaurant,Indian Restaurant
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Baby Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub,Candy Store
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Gym,Clothing Store,Deli / Bodega,Thai Restaurant,Sushi Restaurant,Cosmetics Shop,Pizza Place
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Bakery,Pharmacy,Pet Store,Music Venue,Middle Eastern Restaurant,Bar,Café,Furniture / Home Store,Bank,Supermarket
35,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,8,Pizza Place,Park,Convenience Store,Intersection,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Using the coordenates of Toronto we can create a map and plot each cluster element.

In [38]:
latitude = 43.7166
longitude = -79.3407
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7166, -79.3407.


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(clusters_number)
ys = [i + x + (i*x)**2 for i in range(clusters_number)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(clustered_toronto_pc['Latitude'], clustered_toronto_pc['Longitude'], clustered_toronto_pc['Neighborhood'], clustered_toronto_pc['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Cluster 1:


In [46]:
clustered_toronto_pc.loc[clustered_toronto_pc['Cluster Labels'] == 0, clustered_toronto_pc.columns[[1] + list(range(5, clustered_toronto_pc.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
9,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Hotel,Ramen Restaurant,Pizza Place,Burger Joint
15,Downtown Toronto,0,Coffee Shop,Café,Cosmetics Shop,Cocktail Bar,Gym,Lingerie Store,Farmers Market,Seafood Restaurant,Restaurant,Italian Restaurant
20,Downtown Toronto,0,Coffee Shop,Bakery,Cocktail Bar,Pharmacy,Cheese Shop,Seafood Restaurant,Beer Bar,Farmers Market,Restaurant,Belgian Restaurant
24,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Thai Restaurant,Burger Joint,Korean Restaurant,Indian Restaurant
25,Downtown Toronto,0,Grocery Store,Café,Park,Baby Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub,Candy Store
30,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gym,Clothing Store,Deli / Bodega,Thai Restaurant,Sushi Restaurant,Cosmetics Shop,Pizza Place
31,West Toronto,0,Bakery,Pharmacy,Pet Store,Music Venue,Middle Eastern Restaurant,Bar,Café,Furniture / Home Store,Bank,Supermarket
36,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Restaurant,Brewery,Sporting Goods Shop,Fried Chicken Joint,Scenic Lookout
37,West Toronto,0,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Café,Coffee Shop,Asian Restaurant,Men's Store,Yoga Studio,Miscellaneous Shop


Cluster 2:

In [47]:
clustered_toronto_pc.loc[clustered_toronto_pc['Cluster Labels'] == 1, clustered_toronto_pc.columns[[1] + list(range(5, clustered_toronto_pc.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,Central Toronto,1,Department Store,Food & Drink Shop,Park,Sandwich Place,Breakfast Spot,Hotel,Diner,Discount Store,Distribution Center,Wings Joint


Cluster 3:

In [48]:
clustered_toronto_pc.loc[clustered_toronto_pc['Cluster Labels'] == 2, clustered_toronto_pc.columns[[1] + list(range(5, clustered_toronto_pc.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


##Conclusions
We can see that the different venue kinds are placed almost uniformly around Toronto. On the other hand, there are isolated clusters that are so different between them.